In [ ]:
"""
Create the function that will generate
A number for the site percolation of both random and 
degree centric attacks
"""

In [8]:
import networkx as nx
import numpy as np

In [9]:
def _get_vertex_order(G,selection_type="random"):
    if selection_type == "random":
        return np.random.permutation(list(G.nodes))
    elif selection_type == "degree":
        """ Will organize from highest to lowest degree"""
        degree_dict = dict()
        for k,v in G.degree():
            if v not in degree_dict.keys():
                degree_dict[v] = [k]
            else:
                degree_dict[v].append(k)
        degree_dict

        #get the order of degree
        order_degrees = np.sort(list(degree_dict.keys()))

        node_order = []
        for k in order_degrees:
            node_order += list(np.random.permutation(degree_dict[k]))

        return node_order
    else:
        raise Exception("Invalid Selection Type")
    
    


In [10]:
from tqdm import tqdm

def run_site_percolation(G,vertex_order_type="random",n_iterations=1000):
    total_runs = []

    for y in tqdm(range(0,n_iterations)):
        current_run_results = [0,1]
        """
        1) Start with empty network. Number of clusters, c = 0, currently in network
        Choose at random the order in which vertices will be added to the network
        """

        clusters=dict() #starting out the clusters list as empyt
        vertex_order = _get_vertex_order(G,vertex_order_type)


        """
        2) Add the next vertex in list to the network initially with no edges
        """
        vertex_labels = dict()
        for i,v in enumerate(vertex_order):
            #print(f"Working on vertex {v}")

            """ 2b)
            - increase the cluster count by 1 (because the new vertex is initially a cluster of its own)
            - Make the cluster size of one

            """

            try:
                max_index_plus_1 = np.max(list(clusters.keys())) + 1
                clusters[max_index_plus_1] = 1
                vertex_labels[v] = max_index_plus_1
            except:
                clusters[0] = 1
                vertex_labels[v] = 0
                continue

            """
            3) Go through the edges attached to newly added vertex and add the edges where the other 
            vertex already exists in the network

            4) For each edge added, check if the vertices have the same cluster group number:
            - if yes then do nothing
            - if no, relabel the smaller cluster the same cluster number as the bigger cluster number
            - update the sizes of the 2 clusters from which formed
            """
            already_added_v = set(vertex_order[:i]).intersection(set(G[v].keys()))
            for a_v in already_added_v:
                if vertex_labels[a_v] != vertex_labels[v]:
                    index_max = np.argmax([clusters[vertex_labels[a_v]],clusters[vertex_labels[v]]])
                    if index_max == 0: #need to change all the labels with v
                        replaced_cluster = vertex_labels[v]
                        indexes_to_change = [jj for jj in vertex_labels.keys() if vertex_labels[jj] == vertex_labels[v]]
                        final_cluster = vertex_labels[a_v]
                    else:
                        replaced_cluster = vertex_labels[a_v]
                        indexes_to_change = [jj for jj in vertex_labels.keys() if vertex_labels[jj] == vertex_labels[a_v]]
                        final_cluster = vertex_labels[v]

                    #change the labels
                    for vv in indexes_to_change:
                        vertex_labels[vv] = final_cluster

                    replaced_size = clusters.pop(replaced_cluster)
                    clusters[final_cluster] += replaced_size

            current_run_results.append(np.max([v for v in clusters.values()]))


            #Done adding that vertex and will continue on to next vertex
            #print(f"clusters = {clusters}")

            total_runs.append(current_run_results)
    total_runs = np.array(total_runs)
    
    from scipy.special import comb
    n = len(G.nodes)
    S_r = np.mean(total_runs,axis=0)
    #calculate s_phi : average largest cluster size as a functin of the occupancy probability
    phi = np.arange(0,1.05,0.05)
    r = np.arange(0,n+1,1)
    s_phi = [np.sum([comb(n, r_curr, exact=True)*(phi_curr**r_curr)*((1-phi_curr)**(n- r_curr))*S_r_curr
                        for r_curr,S_r_curr in zip(r,S_r)]) for phi_curr in phi]
    s_phi = np.array(s_phi)/n
    
    return s_phi,phi
    
    

In [26]:
import networkx as nx

def random_degree_site_percolation(G,n_iterations):
    random_degree_site_percolation.stat_names = ["area_above_identity_random_percol",
                                                "area_below_identity_random_percol",
                                                "area_above_identity_degree_percol",
                                                "area_below_identity_degree_percol"]
    s_phi_barabasi_rand,phi_barabasi_rand= run_site_percolation(G,"random",n_iterations)
    s_phi_barabasi_degree,phi_barabasi_degree= run_site_percolation(G,"degree",n_iterations)

    rand_diff = s_phi_barabasi_rand - phi_barabasi_rand
    degree_diff = s_phi_barabasi_degree - phi_barabasi_degree

    dx = phi_barabasi_rand[1]-phi_barabasi_rand[0]

    rand_diff_positive = np.where(rand_diff>0)[0]
    rand_diff_negative = np.where(rand_diff<= 0)[0]
    degree_diff_positive = np.where(degree_diff>0)[0]
    degree_diff_negative = np.where(degree_diff<=0)[0]

    return (np.trapz(rand_diff[rand_diff_positive],dx=dx),
     np.trapz(rand_diff[rand_diff_negative],dx=dx),
     np.trapz(degree_diff[degree_diff_positive],dx=dx),
     np.trapz(degree_diff[degree_diff_negative],dx=dx))


In [27]:
n_iterations = 20
G = nx.random_graphs.watts_strogatz_graph(n=100,p=0.4,k=10)
random_degree_site_percolation(G,n_iterations)

100%|██████████| 20/20 [00:00<00:00, 130.32it/s]


(0.0, -0.013963357497612554, 0.0, -0.043023459125347725)

In [31]:
"stat_names" in dir(random_degree_site_percolation)

False

In [33]:
x = {5:6,10:11}
y = {7:8}
y.update(x)
y

{7: 8, 5: 6, 10: 11}